In [2]:
# 把所有成功的轨迹直接添加到task_DAG目录下json文件的successful_topo字段中
# 读取task_DAG目录下的所有json文件，统计DAG数量以及轨迹数量
import json
trej_cnt = 0
DAG_cnt = 0
for i in range(36079):
    with open(f'D:/data/task_DAG_batch1/{i}.json', 'r', encoding='utf-8') as f:
        response = json.load(f)
    # response ==  {'task_instruction': "Open 'C:\\Users\\user\\Desktop\\office\\The Evolution of Urbanization Rate.xlsx', select all data, and copy it. Then, paste the text from the clipboard, translate it into 'German', and copy the result. Open 'C:\\Users\\user\\Desktop\\office\\pet introduction.docx', paste the translated text from the clipboard, and save it. Meanwhile, open 'C:\\Users\\user\\Desktop\\office\\The Evolution of Urbanization Rate.xlsx', select the 'A' column, and center the content. Finally, open 'C:\\Users\\user\\Desktop\\office\\The Evolution of Urbanization Rate.xlsx' and export it as a PDF document named 'C:\\Users\\user\\Desktop\\pdf\\The Evolution of Urbanization Rate.pdf'.", 'code': "def execute_task():\n    # Step 1: Copy all data from the Excel file\n    table_in_clipboard, xlsx_in_processing = copy_all_data_from_xlsx('C:\\\\Users\\\\user\\\\Desktop\\\\office\\\\The Evolution of Urbanization Rate.xlsx')\n\n    # Step 2: Translate the text in the clipboard to German\n    text_in_clipboard = translate_clipboard_to_language(table_in_clipboard, 'German')\n\n    # Step 3: Paste the translated text into the Word document\n    docx_path, text_in_clipboard = paste_clipboard_to_docx_file('C:\\\\Users\\\\user\\\\Desktop\\\\office\\\\pet introduction.docx', text_in_clipboard)\n\n    # Step 4: Center the content of column A in the Excel file\n    xlsx_in_processing = open_and_center_xlsx_column_content('C:\\\\Users\\\\user\\\\Desktop\\\\office\\\\The Evolution of Urbanization Rate.xlsx', 'A')\n\n    # Step 5: Export the Excel file as a PDF\n    pdf_path, xlsx_in_processing = xlsx_to_pdf_export('C:\\\\Users\\\\user\\\\Desktop\\\\office\\\\The Evolution of Urbanization Rate.xlsx', 'C:\\\\Users\\\\user\\\\Desktop\\\\pdf\\\\The Evolution of Urbanization Rate.pdf')\n\nexecute_task()", 'dag': {'nodes': ['25e2a51e-c019-1a9a-0747-d6fe0e9d457d', '0a4772f6-61ad-49e5-b15e-c02963259484', '8504cc45-066e-d203-cb82-4039ee74ae84', 'a7310aa0-b194-77e3-5c36-996391a1bc7d', 'df3fc68b-fa76-4e19-7da6-aef17792523b'], 'edges': {'25e2a51e-c019-1a9a-0747-d6fe0e9d457d': ['0a4772f6-61ad-49e5-b15e-c02963259484', 'a7310aa0-b194-77e3-5c36-996391a1bc7d', 'df3fc68b-fa76-4e19-7da6-aef17792523b'], '0a4772f6-61ad-49e5-b15e-c02963259484': ['8504cc45-066e-d203-cb82-4039ee74ae84'], '8504cc45-066e-d203-cb82-4039ee74ae84': [], 'a7310aa0-b194-77e3-5c36-996391a1bc7d': [], 'df3fc68b-fa76-4e19-7da6-aef17792523b': []}}}
    # 根据邻接表构建有向无环图并输出拓扑序
    from collections import defaultdict, deque

    # 从response中获取图的信息
    nodes = response['dag']['nodes']
    edges = response['dag']['edges']

    # 构建邻接表和入度表
    graph = defaultdict(list)
    in_degree = defaultdict(int)

    for node in nodes:
        in_degree[node] = 0
        
    for start, end_list in edges.items():
        for end in end_list:
            graph[start].append(end)
            in_degree[end] += 1

    # 使用Kahn算法找出所有拓扑序
    def all_topological_sorts(graph, in_degree, nodes):
        result = []
        
        def backtrack(curr_sequence, remaining_in_degree, available_nodes):
            if len(curr_sequence) == len(nodes):
                result.append(curr_sequence[:])
                return
                
            for node in nodes:
                if remaining_in_degree[node] == 0 and node not in curr_sequence:
                    # 选择当前节点
                    curr_sequence.append(node)
                    
                    # 更新邻居节点的入度
                    neighbor_degrees = {}
                    for neighbor in graph[node]:
                        neighbor_degrees[neighbor] = remaining_in_degree[neighbor]
                        remaining_in_degree[neighbor] -= 1
                    
                    backtrack(curr_sequence, remaining_in_degree, available_nodes)
                    
                    # 回溯
                    curr_sequence.pop()
                    for neighbor, degree in neighbor_degrees.items():
                        remaining_in_degree[neighbor] = degree
        
        backtrack([], in_degree.copy(), set(nodes))
        return result

    with open('D:/data/succ_subtask_traj_ids.json', 'r', encoding='utf-8') as f:
        successful = json.load(f)
    successful_subtasks = []
    for j in successful:
        successful_subtasks.append(j[:-3])

    # 获取所有拓扑序并保存为列表
    topological_sorts = all_topological_sorts(graph, in_degree, nodes)
    all_sorts = []
    for sort in topological_sorts:
        # 检查当前拓扑序中的所有subtask是否都在successful_subtasks中
        if all(subtask in successful_subtasks for subtask in sort):
            all_sorts.append(sort)
    if all_sorts != []:
        trej_cnt += len(all_sorts)
        DAG_cnt += 1
    response["successful_topo"] = all_sorts
    with open(f'D:/data/task_DAG_batch1/{i}.json', 'w', encoding='utf-8') as f:
        json.dump(response, f, indent=4, ensure_ascii=False)
        # print(f"{i}: {all_sorts}")
    # break
print(trej_cnt, DAG_cnt)



328311 17178
